# Loading Libraries

## Setting up the Environment

In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
import rioxarray as rio
from matplotlib.cm import jet

# Filesystem interface to Azure-Datalake Gen1 and Gen2 Storage
import adlfs

# Import Planetary Computer tools
import pystac_client
import planetary_computer


from tqdm import tqdm

# TerraClimate Data

## Loading the Data

We'll use pystac-client to search the Planetary Computer's STAC API for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage.

In [2]:
# Access STAC catalog and collection.
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

collection = catalog.get_collection("terraclimate")
asset = collection.assets["zarr-abfs"]

In [3]:
# Open dataset and remove CRS.
ds = xr.open_dataset(asset.href,**asset.extra_fields["xarray:open_kwargs"])
ds = ds.drop('crs', dim=None) # Remove the CRS coordinate in the dataset

## Trimming the Data

We processes a massive dataset, nearly 2 TB in size, by first trimming it to include only data from November 1, 2017, to November 1, 2019.

In [4]:
# Since this is a HUGE dataset (nearly 2 TB), we should parse the dataset
# Trimming dataset to years 2017 thru 2019
ds = ds.sel(time=slice("2017-11-01", "2019-11-01"))

We then define the geographical bounds for southeastern Australia, specifying the minimum and maximum latitude and longitude values. Boolean masks are created to filter the dataset within these bounds, ensuring that only data within the specified latitude and longitude range is retained.

In [5]:
# Trimming dataset to the desired Lat-Lon bounds (southeastern Australia)

min_lon = 139.94
min_lat = -39.74
max_lon = 151.48
max_lat = -30.92

mask_lon = (ds.lon >= min_lon) & (ds.lon <= max_lon)
mask_lat = (ds.lat >= min_lat) & (ds.lat <= max_lat)

Finally, the dataset is cropped to this smaller region using the where method, effectively reducing its size and focusing on the relevant time period and geographical area for further analysis.

In [6]:
# Crop the dataset to smaller Lat-Lon regions
ds = ds.where(mask_lon & mask_lat, drop=True)

## Producing a GeoTIFF File

Persist the dataset ds in memory to optimize performance.

In [7]:
# Persist dataset in memory.
ds=ds.persist()

Compute the median values along the time dimension, skipping any NaN values, and stores the result in the median variable.

In [8]:
# Compute median along time dimension.
median = ds.median(dim="time", skipna=True).compute()

In [9]:
#Define output file name.
filename = "TerraClimate_output.tiff"

The dimensions of the file are calculated based on the latitude and longitude dimensions of the median dataset. 

In [10]:
# Calculate the dimensions of the file
height = median.dims["lat"]
width = median.dims["lon"]

The Coordinate Reference System (CRS) is set to EPSG:4326, which represents common latitude-longitude coordinates. A transformation is defined using the bounding box to ensure the latitude and longitude information is correctly written to the GeoTIFF. The CRS and transformation are written to the median dataset.

In [11]:
## Define the Coordinate Reference System (CRS) to be common Lat-Lon coordinates
## Define the tranformation using our bounding box so the Lat-Lon information is written to the GeoTIFF
gt = rasterio.transform.from_bounds(min_lon,min_lat,max_lon,max_lat,width,height)
median.rio.write_crs("epsg:4326", inplace=True)
median.rio.write_transform(transform=gt, inplace=True);

The GeoTIFF file is created using the defined parameters, with two bands: one for solar radiation (srad) and one for vapor pressure (vap), and the file is compressed using the LZW algorithm.

In [12]:
# Create the GeoTIFF output file using the defined parameters 
with rasterio.open(
    filename, 'w', driver='GTiff', width=width, height=height,
    crs='epsg:4326', transform=gt, count=14, compress='lzw', dtype='float64') as dst:
    # Write each variable to a corresponding band
    dst.write(median.aet.astype('float64'), 1)      
    dst.write(median['def'].astype('float64'), 2)   
    dst.write(median.pdsi.astype('float64'), 3)     
    dst.write(median.pet.astype('float64'), 4)      
    dst.write(median.ppt.astype('float64'), 5)      
    dst.write(median.q.astype('float64'), 6)        
    dst.write(median.soil.astype('float64'), 7)     
    dst.write(median.srad.astype('float64'), 8)     
    dst.write(median.swe.astype('float64'), 9)      
    dst.write(median.tmin.astype('float64'), 10)    
    dst.write(median.tmax.astype('float64'), 11)    
    dst.write(median.vap.astype('float64'), 12)     
    dst.write(median.vpd.astype('float64'), 13)     
    dst.write(median.ws.astype('float64'), 14)     